In [1]:
import ipywidgets as ipw
from astropy.table import Table
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

#  You will need to change some values in the cell below 

In [2]:
comp_locations = Table.read('aperture_locations (1).csv')
flux_table = Table.read('TIC-467615239-2021-09-28-relative-flux.csv')

In [12]:
flux_table = flux_table[flux_table['filter']=='ip']

In [13]:
present_ids = sorted(set(flux_table['star_id']))

star_type = {sid: desc for sid, desc in comp_locations['star_id', 'marker name']}
    
select_multi = ipw.SelectMultiple(options=[(f'{sid:3d} {star_type[sid]}', sid) for sid in present_ids],
                                  description='For these objects')
select_multi.rows = 10

columns = sorted(flux_table.colnames)

y_axis = ipw.Dropdown(options=columns, description="Graph this")
y_axis.value = 'relative_flux'

y_offset = ipw.FloatSlider(min=0, max=1, value=0, step=0.01, description='Offset by')

detrend_by = ipw.Dropdown(options=columns, description='Detrend by', value=None)

title = ipw.Text(description='Title')

In [14]:
plt.ioff()

fig, ax = plt.subplots()
fig.canvas.header_visible = False
ax.grid()

ax.set_xlabel('BJD')
ax.set_ylabel('Flux');

In [15]:
def linear_detrend(data, parameter):
    def f(x, slope, intercept):
        return slope * x + intercept

    popt, pcov = curve_fit(f, parameter, data)
    
    return data - popt[0] * parameter

In [16]:
def update_plot(change):
    ax.lines.clear()
    min = 100
    max = -100
    data_col = y_axis.value
    for idx, star_id in enumerate(select_multi.value):
        data = flux_table[flux_table['star_id'] == star_id]
        
        norm_flux = data[data_col] / data[data_col].mean()
        if detrend_by.value:
            print(detrend_by.value)
            norm_flux = linear_detrend(norm_flux, data[detrend_by.value])
            norm_flux = norm_flux / norm_flux.mean()
            
        norm_flux += y_offset.value * idx

        ax.plot(data['BJD'], norm_flux, label=f'{star_id}')
        #lines[0].set_data(data['BJD'], norm_flux)
        ax.set_xlim(data['BJD'].min(), data['BJD'].max())  
        min = norm_flux.min() if norm_flux.min() < min else min
        max = norm_flux.max() if norm_flux.max() > max else max
        
    ax.set_ylim(0.8 * min, 1.1 * max)
    ax.set_xlabel('BJD')
    ax.set_ylabel(data_col)
    if title.value:
        ax.set_title(title.value)
    ax.legend()
    fig.canvas.draw()
    fig.canvas.flush_events()

In [17]:
vb = ipw.VBox(children=[y_axis, select_multi, y_offset, detrend_by, title])

for wid in vb.children:
    wid.observe(update_plot, names='value')

In [18]:
ipw.AppLayout(
    center=fig.canvas,
    right_sidebar=vb,  # select_multi,
    pane_heights=[0, 6, 1]
)

AppLayout(children=(VBox(children=(Dropdown(description='Graph this', index=26, options=('BJD', 'Dec', 'RA', '…

In [10]:
start = 2459910.8122
end   = 2459910.9032
low = 0.8
high = 2.0
plt.vlines(start, low, high, colors='r', linestyle='--', alpha=0.5)
plt.vlines(end, low, high, colors='r', linestyle='--', alpha=0.5)
plt.text(start, low+0.0005, f'Predicted\nIngress\n{start-2400000-int(start - 2400000):.3f}', horizontalalignment='center',c='r')
plt.text(end, low+0.0005, f'Predicted\nEgress\n{end-2400000-int(end - 2400000):.3f}', horizontalalignment='center',c='r')
fig.canvas.draw()

In [11]:
ax.set_title('TIC 467615239.01 UT2022.11.27\n Paul P. Feder Observatory 0.4m (ip filter,120 exp,fap 11-21-26)')
       
fig.canvas.draw()